In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pydub

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from pydub import AudioSegment
import wave
import numpy as np
import matplotlib.pyplot as plt
import librosa
import scipy.signal


In [20]:
# ボイスメモで収録したファイルを読み込む
input_path = '/content/drive/My Drive/IVRC/rain.mp3'
sounds = AudioSegment.from_file(input_path, 'mp3')

/Users/ishikawaritsuki/.pyenv/versions/3.10.6/lib/python3.10/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe'

In [6]:
# 基本情報の表示
#　　音声波形の表示
#print(f'channel: {sounds.channels}')
#print(f'frame rate: {sounds.frame_rate}')
#print(f'duration: {sounds.duration_seconds} s')
# チャンネルが2 （ステレオ) の場合，交互にデータが入っているので，二つおきに読み出す。
# ただし，今回の場合はモノラルのはず。つまり，sounds.channels = 1
sig = np.array(sounds.get_array_of_samples())[::sounds.channels]
dt = 1.0/sounds.frame_rate # サンプリング時間
# 時間アレイを作る
tms = 0.0 # サンプル開始時間を0にセット
tme = sounds.duration_seconds # サンプル終了時刻
tm = np.linspace(tms, tme, len(sig), endpoint=False) # 時間ndarrayを作成
# DFT
N = len(sig)
X = np.fft.fft(sig)
f = np.fft.fftfreq(N, dt) # Xのindexに対応する周波数のndarrayを取得
# データをプロット
'''
fig, (ax01, ax02) = plt.subplots(nrows=2, figsize=(12, 8))
plt.subplots_adjust(wspace=0.0, hspace=0.6)
ax01.set_xlim(tms, tme)
ax01.set_xlabel('time (s)')
ax01.set_ylabel('x')
#ax01.plot(tm, sig) # 入力信号
ax02.set_xlim(0, 2000)
ax02.set_xlabel('frequency (Hz)')
ax02.set_ylabel('|X|/N')
#ax02.plot(f[0:N//2], np.abs(X[0:N//2])/N) # 振幅スペクトル
#plt.show()
'''

NameError: name 'sounds' is not defined

In [7]:
# 波形カット
fft_size = 1024
plt.figure()
hh = ([i for i, x in enumerate(sig) if x > 20])
st=(hh[0])
ed=(hh[-1])
jj = (ed-st)%fft_size
jj = fft_size - jj
ed = ed + jj
#print(ed-st)
frame = sig[st:ed].copy()
time2 = np.arange(0, ed-st)
#plt.plot(time2,frame)

NameError: name 'sig' is not defined

<Figure size 640x480 with 0 Axes>

In [8]:
# プリエンファシス（高周波強調）
p = 0.97
time3 = np.arange(0, fft_size)
target_count = 0
frame1 = frame[target_count*fft_size:(target_count+1)*fft_size].copy()
frame1 = scipy.signal.lfilter([1.0, -p], 1, frame1)
frame1[0] = frame1[1]
#plt.plot(time3, frame1)

NameError: name 'frame' is not defined

In [9]:
# ハミング窓を波形にかける
# plt.figure()
frame = frame1 * np.hamming(fft_size)
#plt.plot(time3, frame1)
#plt.plot(time3, frame)

NameError: name 'frame1' is not defined

In [10]:
# フーリエ変換
# plt.figure()
time4 = np.arange(0, fft_size/2)
spectrum = np.fft.fft(frame1)
spectrum = spectrum[: int(fft_size/2)]
#plt.plot(time4, spectrum)

NameError: name 'frame1' is not defined

In [11]:
# 絶対値とログをかける
#plt.figure()
rspectrum = np.fft.ifft(spectrum)
#plt.plot(rspectrum)


# fig = plt.figure()
# ax = fig.add_subplot()
spectrum = (np.abs(spectrum))
spectrum = 10*np.log10(spectrum**2)
#ax.plot(time4, spectrum)

NameError: name 'spectrum' is not defined

In [12]:
# メルフィルタをかける
from librosa.core.convert import mel_frequencies
#plt.figure()
mel_basis = librosa.filters.mel(sr=44100, n_fft=1024-1, n_mels=20, fmax=44100//2)
freqencies = librosa.fft_frequencies(sr=44100, n_fft=1024-1)
# for i in range(20):
  #plt.plot(freqencies, mel_basis[i])

#plt.figure()
mel_amplitude_spectrum = np.dot(mel_basis, spectrum)
change = mel_amplitude_spectrum
mel_amplitude_spectrum = np.log(change)
mel_frequencies = librosa.mel_frequencies(n_mels=20+2, fmin=0.0, fmax=44100/2, htk=True)
mel_frequencies = mel_frequencies[1:-1]
#plt.plot(mel_frequencies, mel_amplitude_spectrum)


NameError: name 'spectrum' is not defined

In [ ]:
# 離散コサイン変換
from matplotlib.figure import NonGuiException
# plt.figure()
time4 = np.arange(0, 20)
nceps=12
spec = scipy.fftpack.realtransforms.dct(mel_amplitude_spectrum, type=2, norm="ortho", axis=-1)
speck=spec[:nceps]
#plt.xlim([0, 20])
#plt.plot(time4, spec)
#plt.plot(speck)

<ipython-input-13-4b652553d1bf>:6: DeprecationWarning: Please use `dct` from the `scipy.fftpack` namespace, the `scipy.fftpack.realtransforms` namespace is deprecated.
  spec = scipy.fftpack.realtransforms.dct(mel_amplitude_spectrum, type=2, norm="ortho", axis=-1)


In [ ]:
# 逆フーリエ変換
# plt.figure()
qefr_axis = np.arange(0, fft_size/2)
cepstrum = np.fft.ifft(spectrum)
#plt.plot(qefr_axis, np.real(cepstrum))

In [ ]:
# 高域部分カットして再フーリエ変換
# plt.figure()
cepstrum_low = cepstrum.copy()
cep_threshold = 33
cepstrum_low[(cep_threshold+1):-(cep_threshold)] = 0.0
#plt.plot(qefr_axis, np.real(cepstrum_low))
log_power_ceplo = np.abs(np.fft.fft(cepstrum_low))

In [ ]:
print(qefr_axis.tolist())
print(log_power_ceplo.tolist())

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158

In [13]:
# -*- coding: utf-8 -*-
# 波形を近似
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import ntpath
from sklearn import linear_model
import joblib
from scipy import optimize

# 近似関数
def approx_func(param, x, y):
    y = param[0] * x ** 5 + param[1] * x ** 4 + param[2] * x ** 3 + param[3] * x ** 2 + param[4] * x + param[5]
    return y

def residual_func(param, x, y):
    residual = y - approx_func(param, x, y)
    return residual

# 決定係数R^2の計算
def calc_r2(x_data, y_data, y_predict):
    residuals = y_data - y_predict
    rss = np.sum(residuals**2)  # residual sum of squares = rss
    tss = np.sum((y_data - np.mean(y_data)) ** 2)  # total sum of squares = tss
    r2 = 1 - (rss / tss)
    return r2

# 開始時間～終了時間
(time_start, time_end) = (0, 580)

# パラメータの初期値
param1 = [0, 0, 0, 0, 0, 0]


# 空のデータフレームを作成
df = pd.DataFrame({})

# 電流値の列データを取り出し
Its = log_power_ceplo


# 経過時間の列データを取り出し
times = qefr_axis

Its = Its[(times > time_start) & (times < time_end)]
times = times[(times > time_start) & (times < time_end)]

# 近似直線のパラメータを計算
lq = optimize.leastsq(residual_func, param1, args=(times, Its))  # 係数
(a1, a2, a3, a4, a5, b) = (lq[0][0], lq[0][1], lq[0][2], lq[0][3], lq[0][4], lq[0][5])

# 近似直線
y_predict = approx_func(lq[0], times, Its)

# 決定係数
r2 = calc_r2(times, Its, y_predict)

# パラメータの表示
print("回帰係数a1:", a1)
print("回帰係数a2:", a2)
print("回帰係数a3:", a3)
print("回帰係数a4:", a4)
print("切片:", b)
print("決定係数:", r2)

# 電流値をプロット
plt.rcParams["font.family"] = "DejaVu Serif"
plt.plot(times, Its, lw=1, c="r", alpha=0.7, ms=2)

# 電流値をプロット
plt.plot(times, y_predict, lw=1, c="b", alpha=0.7, ms=2)



NameError: name 'log_power_ceplo' is not defined

In [ ]:
print(times.tolist())
print(y_predict.tolist())

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158.0, 1

In [ ]:
# 母数を削減してグラフを粗くすることでデータ量削減
audio_time=[]
audio_level=[]
for i in range(len(times.tolist())):
  if i % 100==0:
    audio_time.append(times.tolist()[i])
    audio_level.append(y_predict.tolist()[i])

In [ ]:
len(audio_time)

6

In [14]:
plt.plot(np.array(audio_time), np.array(audio_level), lw=1, c="b", alpha=0.7, ms=2)
plt.plot(times, y_predict, lw=1, c="r", alpha=0.7, ms=2)

NameError: name 'audio_time' is not defined

In [15]:
# 波形データのスケール変換(波形の最大・最小をモーターの最大・最小速度へ変換)
#i(モータの速度)の範囲　100<i<255
motor_max = 255
motor_min = 100
audio_max = max(audio_level)
audio_min = min(audio_level)
dif_motor = motor_max - motor_min
dif_audio = audio_max - audio_min
audio_level_scalechange = []
for level in audio_level:
  audio_level_scalechange.append(motor_min + ((level - audio_min)/(audio_max - audio_min)) * (motor_max - motor_min))
#print("max:{}, min:{}".format(audio_max, audio_min))

NameError: name 'audio_level' is not defined

In [38]:
# audioのプログラム生成
count = 0

for i in range(len(audio_time)-1):
  if audio_level_scalechange[i] <= audio_level_scalechange[i+1]:
    j = audio_level_scalechange[i]
    while(j<audio_level_scalechange[i+1]):
      j+=((audio_level_scalechange[i+1]-audio_level_scalechange[i])/(motor_max-motor_min))
      count+=1
  else:
    j = audio_level_scalechange[i]
    while(j>audio_level_scalechange[i+1]):
      j-=((audio_level_scalechange[i]-audio_level_scalechange[i+1])/(motor_max-motor_min))
      count+=1

count=count/1000
for i in range(len(audio_time)-1):
  if audio_level_scalechange[i] <= audio_level_scalechange[i+1]:
    print("for(i={};i<{};i+={})".format(audio_level_scalechange[i],audio_level_scalechange[i+1],((audio_level_scalechange[i+1]-audio_level_scalechange[i])/(motor_max-motor_min))) + "{")
    print("    analogWrite(PIN_IN12, i);")
    print("    analogWrite(PIN_IN21, i);")
    print("    delay({});".format(tme/count))
    print("}")
  else:
    print("for(i={};i>{};i-={})".format(audio_level_scalechange[i],audio_level_scalechange[i+1],((audio_level_scalechange[i]-audio_level_scalechange[i+1])/(motor_max-motor_min))) + "{")
    print("    analogWrite(PIN_IN12, i);")
    print("    analogWrite(PIN_IN21, i);")
    print("    delay({});".format(tme/count))
    print("}")

print("Serial.println(\"終了\");")

for(i=100.0;i<255.0;i+=1.0){
    analogWrite(PIN_IN12, i);
    analogWrite(PIN_IN21, i);
    delay(77.29151892417198);
}
for(i=255.0;i>246.6430448338958;i-=0.05391583978131742){
    analogWrite(PIN_IN12, i);
    analogWrite(PIN_IN21, i);
    delay(77.29151892417198);
}
for(i=246.6430448338958;i>231.52769667243135;i-=0.09751837523525454){
    analogWrite(PIN_IN12, i);
    analogWrite(PIN_IN21, i);
    delay(77.29151892417198);
}
for(i=231.52769667243135;i>201.50494875910164;i-=0.19369514782793362){
    analogWrite(PIN_IN12, i);
    analogWrite(PIN_IN21, i);
    delay(77.29151892417198);
}
for(i=201.50494875910164;i>122.01364530615125;i-=0.5128471190512929){
    analogWrite(PIN_IN12, i);
    analogWrite(PIN_IN21, i);
    delay(77.29151892417198);
}
Serial.println("終了");
